In [35]:
#dvir amitay 


In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV


file_path = r'C:\Users\dvir\Desktop\שנה ג\סמסטר ב\כרייה וניתוח נתונים\מטלות\מטלה 2\dataset.csv'
df = pd.read_csv(file_path)

df

,manufactor,Year,model,Hand,Gear,capacity_Engine,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test,Supply_score
0,יונדאי,2015,i35,2,אוטומטית,1600,בנזין,פרטית,פרטית,רעננה - כפר סבא,רעננה,51000.0,2.0,11/07/2023,11/07/2023,['רכב שמור בקנאות\nמוכרת עקב קבלת רכב חברה'],כחול כהה מטאלי,144000,NaN,NaN
1,ניסאן,2018,ניסאן מיקרה,1,אוטומטית,1200,בנזין,פרטית,פרטית,מושבים בשרון,אבן יהודה,49000.0,0.0,06/04/2022,22/05/2022,['שמורה כל התוספות'],כחול בהיר,69000,NaN,NaN
2,סוזוקי,2010,סוזוקי סוויפט,1,אוטומטית,1450,בנזין,NaN,NaN,רמת,רמת,22500.0,1.0,29/10/2022,29/10/2022,['רכב במצב מתוחזק ברמה גבוהה טסט עד אפריל 2023'],None,145000,NaN,NaN
3,טויוטה,2016,אוריס,1,טיפטרוניק,1600,בנזין,פרטית,פרטית,נס ציונה - רחובות,רחובות,63000.0,5.0,16/05/2024,16/05/2024,['אוטו במצב חדש!! שמור בקנאות!! נהג יחיד מטופל...,אפור מטאלי,27300,NaN,NaN
4,קיה,2012,פיקנטו,1,אוטומטית,1248,בנזין,NaN,NaN,"ראשל""צ והסביבה",ראשון לציון,37000.0,1.0,13/06/2022,13/06/2022,['שמור'],NaN,70000,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,סקודה,2016,סקודה אוקטביה (2016),2,אוטומטית,"1,400",בנזין,NaN,NaN,ראש העין והסביבה,כפר קאסם,60000.0,1.0,18/07/2022,18/07/2022,['רכב שמור &lt;br/&gt;&lt;br/&gt;פירוט:&lt;br/...,NaN,180,NaN,NaN
1496,אלפא רומיאו,2013,אלפא רומיאו ג'ולייטה,4,ידנית,1400,בנזין,NaN,NaN,חיפה וחוף הכרמל,חיפה,45000.0,0.0,44791,44791,['פרטית \r\nמכונית יפיפיה ללא שום תאונות ואו ת...,NaN,160000,NaN,NaN
1497,סקודה,2014,סקודה ראפיד (2014),1,אוטומטית,"1,400",בנזין,NaN,NaN,NaN,כפר מנדא,30000.0,1.0,30/08/2022,30/08/2022,['רכב נקי גיר שילדה מנוע במצב מעולה &lt;br/&gt...,NaN,NaN,NaN,NaN
1498,ניסאן,2011,ניסאן ג'וק JUKE,3,אוטומטית,1600,בנזין,פרטית,פרטית,אשדוד - אשקלון,אשדוד,28000.0,0.0,31/08/2023,09/11/2023,"['רכב מתוחזק היטב ללא תאונות או תקלות, טסט עד ...",אפור מטאלי,118000,81,NaN


In [24]:
def fill_missing_values(df):
    # מילוי ערכים חסרים
    # קילומטראז לפי כמה שנים הרכב על הכביש *15000 (ממוצע שנתי בישראל)
    df['Km'] = df.apply(lambda row: (2024 - row['Year']) * 15000 if pd.isna(row['Km']) else row['Km'], axis=1)
    
    # בשביל נפח מנוע נמצא את כל הרכבים מאותו דגם ונמלא את הערך השכיח ביותר. אם אין עוד רכבים מדגם זה, נמלא "0"
    def mode(series):
        return series.mode().iloc[0] if not series.mode().empty else np.nan
    
    df['capacity_Engine'] = df.groupby(['manufactor', 'Year', 'model'])['capacity_Engine'].transform(lambda x: x.fillna(mode(x)))
    df['capacity_Engine'] = df.groupby('model')['capacity_Engine'].transform(lambda x: x.fillna(mode(x)))
    df['capacity_Engine'].fillna('0', inplace=True)
    df['capacity_Engine'] = df['capacity_Engine'].astype(str).str.replace(',', '').str.replace(' ', '')
    df['capacity_Engine'] = df['capacity_Engine'].astype(int)
    
    df['Gear'].fillna('Unknown', inplace=True)
    df['Engine_type'].fillna('Unknown', inplace=True)
    
    return df

In [25]:
def create_new_columns(df):
    # יצירת עמודות חדשות רלוונטיות
    # ניצור עמודה חדשה בשם "גיל הרכב" על מנת שיהיה יותר נוח לעבוד עליה מאשר שנת ייצור
    df['Car_age'] = 2024 - df['Year']    
    # המרת עמודת Km למחרוזת, הסרת תווים לא רצויים והמרה בחזרה ל-int
    df['Km'] = df['Km'].astype(str).str.replace(',', '').str.replace(' ', '')
    df['Km'] = pd.to_numeric(df['Km'], errors='coerce')
    # מילוי ערכי NaN בעמודת Km לפי מספר השנים שהרכב על הכביש * 15,000 (ממוצע שנתי בישראל)
    df['Km'] = df.apply(lambda row: (2024 - row['Year']) * 15000 if pd.isna(row['Km']) else row['Km'], axis=1)
    # המרת עמודת Km לסוג int לאחר המילוי
    df['Km'] = df['Km'].astype(int)
    # הכפלת ערכי הקילומטר במקרה שהם נמוכים מ-1000 וגיל הרכב גדול מ-1
    df.loc[(df['Km'] < 1000) & (df['Car_age'] > 1), 'Km'] *= 1000
    # יצירת העמודה החדשה Average_km_per_year
    df['Average_km_per_year'] = df['Km'] / df['Car_age']
    # טיפול במקרים בהם Car_age הוא 0 (כדי למנוע חלוקה באפס)
    df.loc[df['Car_age'] == 0, 'Average_km_per_year'] = 0
    # עיגול הערכים בעמודת Average_km_per_year למספר השלם הקרוב ביותר
    df['Average_km_per_year'] = df['Average_km_per_year'].round().astype(int)
    
    return df


In [26]:
def reduce_data(df):
    # צמצום הדאטה - השארת עמודות רלוונטיות בלבד
    columns_to_keep = ['manufactor', 'model', 'Hand', 'Gear', 'capacity_Engine', 'Engine_type', 'Km', 'Car_age', 'Average_km_per_year', 'Price']
    df = df.filter(columns_to_keep)
     # יצירת משתני דמה (dummies) עבור העמודות הקטגוריאליות
    df = pd.get_dummies(df, columns=['manufactor', 'model', 'Gear', 'Engine_type'], drop_first=True)
    
    return df

In [32]:
def perform_elastic_net_with_cv(df, param_grid):
    X = df.drop(columns=['Price'])
    y = df['Price']
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    elastic_net = ElasticNet(random_state=42)
    grid_search = GridSearchCV(elastic_net, param_grid, cv=10, scoring='neg_mean_squared_error')
    grid_search.fit(X_scaled, y)
    
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    
    neg_mse_scores = cross_val_score(best_model, X_scaled, y, cv=10, scoring='neg_mean_squared_error')
    
    mse_scores = -neg_mse_scores
    rmse_scores = np.sqrt(mse_scores)
    mean_rmse = np.mean(rmse_scores)
    
    print(f"Best parameters: {best_params}")
    print(f"Best RMSE: {mean_rmse}")
    
    return best_model


In [33]:
def prepare_data(df):
    df = fill_missing_values(df)
    df = create_new_columns(df)
    df = reduce_data(df)
    param_grid = {'alpha': [0.1, 0.5, 1.0, 5.0, 10.0],'l1_ratio': [0.1, 0.5, 0.7, 0.9]}
    model = perform_elastic_net_with_cv(df, param_grid)
    return df, model

In [34]:
df_prepared, model = prepare_data(df)

Best parameters: {'alpha': 0.5, 'l1_ratio': 0.7}
Best RMSE: 13555.868034974828
